In [8]:
import os
import sys
import csv
import requests
import json
import operator
import time
import datetime
import platform
import itertools

In [9]:
# function to write out text files
def writeText(text, path, mode = 'a'):
    with open (path, mode, encoding = 'utf-8') as textout:
        textout.write((text))
        
# function to write out Json files
def writeJson(json, path, mode = 'w'): 
    with open(path, mode) as file:
        file.write(json.dumps(json))
        
# function to wrrite out csv files
def writeCsv(listOut, outputFile):         
    import csv
    with open (outputFile, "w", newline='', encoding = 'utf-8') as outputfile:
        writer = csv.writer(outputfile, delimiter = ",")
        for element in listOut:
            writer.writerow(element)

In [10]:
# function to get text files as strings
def getTxt(path):  
    return open(path, 'r').read()

# import data from prep_out.csv
def getCsv(path, delim = ','): 
    list_return = []
    with open (path, encoding = 'utf-8') as file:
        csvreader = csv.reader(file, delimiter = delim)        
        for i, line in enumerate(csvreader):
            list_inner = []
            list_inner.append(line[0].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_inner.append(line[1].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_inner.append(line[2].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_return.append(list_inner)
    
    
    return list_return

In [11]:
# platform specific paths for opening prep_out.csv
if platform.system() == 'Windows':
    prep_data = getCsv('..\\output\\prep_out_300.csv')
else:
    prep_data = getCsv('../output/prep_out_300.csv')

In [12]:
# testing imported data
print(len(prep_data))   
print(prep_data[0])

3573
[['domestic', ' general', ' government', ' health', ' expenditure', ' capita', ' current', ' us', ' $', ' countries'], ['public', ' expenditure', ' health', ' domestic', ' sources', ' capita', ' expressed', ' current', ' us', ' dollars'], ['social issues', ' politics', ' environment', ' national accounts', ' health systems by country', ' military budget']]


## Word2Vec

In [13]:
# function for summing up single vectors (=words) in a vectorlist
def sumVect(list_vect, ind):
    try:
        for i, elem in enumerate(list_vect):
            if not elem:
                del list_vect[i]
        
        int_len_vect = len(list_vect[0])
        int_len_desc = len(list_vect)
        list_vect_sum = [0] * int_len_vect
        for vect in list_vect:
            for i, dim in enumerate(vect):
                list_vect_sum[i] += float(dim)
        
        for i, vect in enumerate(list_vect_sum):
            list_vect_sum[i] = float(float(vect)/int_len_desc)
            
            
        return(list_vect_sum)
    except Exception as e:
        print(ind)
        print(list_vect)
        print(i)
        print(list_vect_sum)
        print(e)

In [14]:
# creating list of vectors by accessing a Word2Vec model set up on the WU server
def getVecList(list_in):
    failcnt = 0
    failelemcnt = 0
    str_log = 'Started at:\n' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S') + '\n'
    print('Started at:')
    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'),'\n')
    # List of Lists as input needed
    adress = 'http://word2vec.ai.wu.ac.at/googlenews/model?word='
    # Alternative model:
#     adress = 'http://word2vec.ai.wu.ac.at/twitter27_200/model?word='
    # Alternative model:
#     adress = 'http://word2vec.ai.wu.ac.at/glove840_300/model?word='
    # Alternative model:
#     adress = 'http://word2vec.ai.wu.ac.at/ft_en_300/model?word='
    list_return = [[],[]]
    list_failed = [] 
    
    for i, elem in enumerate(list_in):
        inner_feat, inner_label = [], []
        str_1 = '-------------------------------\n' + 'Element ' + str(i)
        #print(str_1)
        str_log += str_1 + '\n'
    
#         print(elem)
        for str_elem in elem[1]:
            elem[0].append(str_elem)
        del elem[1]
        
# name and desc ----------------------------------------
        for str_elem in elem[0]:
            word = str_elem.strip() 
            try:
                inner_feat.append([float(x) for x in 
                                   requests.get((adress+word)).text.replace(' ','').replace('[','').replace(']','').split(',')])
            except:
                try:
                    inner_feat.append([float(x) for x in 
                                       requests.get((adress+word.title())).text.replace(' ','').replace('[','').replace(']','').split(',')])
                except Exception as e:
                    str_2 = 'fail\n' + adress+(word) + '\n' + str(e) + '\n' + requests.get((adress+word)).text
                    str_log += str_2 + '\n'
                    inner_feat.append([])
                    list_failed.append(word)
                    failcnt += 1

# label ----------------------------------------
        for str_elem in elem[1]:
            if str_elem:
                word = str_elem.strip()
                inner_label.append(str_elem)

        
# create output        
        if (inner_feat and inner_label):
            list_return[0].append(sumVect(inner_feat, i))
            list_return[1].append(inner_label)
        else:
            str_log += 'fail\nEmpty List can not be appended!\n'
            failelemcnt += 1
            
        
        
    print('Failed words: ', failcnt)
    print('Failed Elements: ', failelemcnt)
    str_log += '\nFinished at:\n' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
    writeText(str_log, '../output/log.txt')    
    print('\nEnded at:')
    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')) 
    return list_return, list_failed

In [15]:
# call functio for creating vector list
# Failed words: words which could not be interpreted by the model, e.g. 'rdf-idf'
# Failed elements: elements/datasets which are containing ether no features or no labels
vect_list = getVecList(prep_data)

Started at:
2019-01-13 16:34:50 

Failed words:  2733
Failed Elements:  35

Ended at:
2019-01-13 16:48:19


In [9]:
len(vect_list[0][0])

3538

In [10]:
features = vect_list[0][0]
labels = vect_list[0][1]
missing = vect_list[1]

In [11]:
# create output data which can be imported by the ML notebook
if platform.system() == 'Windows':
    writeCsv(features, '..\\output\\feat.csv')
    writeCsv(labels, '..\\output\\labels.csv')
    writeCsv(missing, '..\\output\\missing_words.csv')
else:
    writeCsv(features, '../output/feat.csv')
    writeCsv(labels, '../output/labels.csv')
    writeCsv(missing, '../output/missing_words.csv')

In [12]:
print(features[0])
print('-----------------------------------')
print(labels[0])

[-0.020310211181640624, 0.01893463134765625, 0.02904205322265625, 0.11624755859375, -0.11915283203125, -0.0532867431640625, 0.035272216796875, -0.09762725830078126, 0.11873779296875, -0.016156005859375, -0.071173095703125, -0.0776611328125, -0.059136962890625, 0.070416259765625, -0.19630126953125, 0.126226806640625, -0.02576904296875, 0.0814300537109375, -0.020050048828125, 0.09813232421875, -0.031011962890625, -0.06298828125, 0.0118133544921875, -0.01236572265625, 0.036767578125, 0.01748046875, -0.1005615234375, 0.09871978759765625, -0.05386962890625, -0.0214080810546875, 0.00439910888671875, -0.1583221435546875, 0.0052215576171875, -0.0090240478515625, -0.03025360107421875, -0.1127349853515625, -0.007860565185546875, 0.073138427734375, 0.02081451416015625, 0.091009521484375, 0.06494140625, 0.00750274658203125, 0.05267333984375, -0.01016845703125, -0.035058212280273435, -0.0870635986328125, -0.028472900390625, 0.074761962890625, -0.15401611328125, -0.02520599365234375, 0.1189773559570